In [1]:
# !wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
# !unzip multi_cased_L-12_H-768_A-12.zip

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np
import tensorflow as tf
import pandas as pd
from tqdm import tqdm

In [4]:
import json

with open('dataset.json') as fopen:
    data = json.load(fopen)
    
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

In [5]:
BERT_VOCAB = 'multi_cased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'multi_cased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'multi_cased_L-12_H-768_A-12/bert_config.json'

tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=False)

In [6]:
GO = 101
EOS = 102

In [7]:
from unidecode import unidecode

def get_inputs(x, y):
    input_ids, input_masks, segment_ids, ys = [], [], [], []
    for i in tqdm(range(len(x))):
        tokens_a = tokenizer.tokenize(unidecode(x[i]))
        tokens_b = tokenizer.tokenize(unidecode(y[i]))
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        
        segment_id = [0] * len(tokens)
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_id)

        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        
        r = tokenizer.convert_tokens_to_ids(tokens_b + ["[SEP]"])
        if len([k for k in r if k == 0]):
            print(y[i], i)
            break
        
        ys.append(r)
        
    return input_ids, input_masks, segment_ids, ys

In [8]:
train_input_ids, train_input_masks, train_segment_ids, train_Y = get_inputs(train_X, train_Y)

100%|██████████| 200000/200000 [02:25<00:00, 1372.95it/s]


In [9]:
test_input_ids, test_input_masks, test_segment_ids, test_Y = get_inputs(test_X, test_Y)

100%|██████████| 5000/5000 [00:03<00:00, 1448.79it/s]


In [10]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
epoch = 20
batch_size = 16
warmup_proportion = 0.1
num_train_steps = len(train_input_ids)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [11]:
from tensor2tensor.utils import beam_search
import bert_decoder as modeling_decoder

class Model:
    def __init__(
        self,
        learning_rate = 2e-5,
        training = True,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        def forward(x, segment, masks, y, reuse = False, config = bert_config):
            
            with tf.variable_scope('bert',reuse=reuse):
        
                model = modeling.BertModel(
                    config=config,
                    is_training=training,
                    input_ids=x,
                    input_mask=masks,
                    token_type_ids=segment,
                    use_one_hot_embeddings=False)
                memory = model.get_sequence_output()
            
            with tf.variable_scope('bert',reuse=True):
                Y_seq_len = tf.count_nonzero(y, 1, dtype=tf.int32)
                y_masks = tf.sequence_mask(Y_seq_len, tf.reduce_max(Y_seq_len), dtype=tf.float32)
                
                model = modeling_decoder.BertModel(
                    config=config,
                    is_training=training,
                    input_ids=y,
                    input_mask=y_masks,
                    memory = memory,
                    memory_mask = masks,
                    use_one_hot_embeddings=False)
                output_layer = model.get_sequence_output()
                embedding = model.get_embedding_table()
                
            with tf.variable_scope('cls/predictions',reuse=reuse):
                with tf.variable_scope('transform'):
                    input_tensor = tf.layers.dense(
                    output_layer,
                    units = config.hidden_size,
                    activation = modeling.get_activation(bert_config.hidden_act),
                    kernel_initializer = modeling.create_initializer(
                        bert_config.initializer_range
                    ),
                )
                input_tensor = modeling.layer_norm(input_tensor)

                output_bias = tf.get_variable(
                'output_bias',
                shape = [bert_config.vocab_size],
                initializer = tf.zeros_initializer(),
                )
                logits = tf.matmul(input_tensor, embedding, transpose_b = True)
                return logits

        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        
        self.training_logits = forward(self.X, self.segment_ids, self.input_masks, decoder_input)
        print(self.training_logits)

        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        initial_ids = tf.fill([batch_size], GO)
        
        def symbols_to_logits(ids):
            x = tf.contrib.seq2seq.tile_batch(self.X, 1)
            segment = tf.contrib.seq2seq.tile_batch(self.segment_ids, 1)
            masks = tf.contrib.seq2seq.tile_batch(self.input_masks, 1)
            logits = forward(x, segment, masks, ids, reuse = True)
            return logits[:, tf.shape(ids)[1]-1, :]
        
        final_ids, final_probs, _ = beam_search.beam_search(
            symbols_to_logits,
            initial_ids,
            1,
            tf.reduce_max(self.X_seq_len),
            bert_config.vocab_size,
            0.0,
            eos_id = EOS)
        
        self.fast_result = final_ids
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')

In [12]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Tensor("cls/predictions/MatMul:0", shape=(?, ?, 119547), dtype=float32)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.


In [13]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if 'bert/' + name in name_to_variable:
            assignment_map[name] = name_to_variable['bert/' + name]
            initialized_variable_names[name] = 1
            initialized_variable_names[name + ':0'] = 1
        elif name in name_to_variable:
            assignment_map[name] = name_to_variable[name]
            initialized_variable_names[name] = 1
            initialized_variable_names[name + ':0'] = 1
        

    return (assignment_map, initialized_variable_names)

In [14]:
tvars = tf.trainable_variables()

checkpoint = BERT_INIT_CHKPNT
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [15]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from multi_cased_L-12_H-768_A-12/bert_model.ckpt


In [16]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [17]:
from tqdm import tqdm
import time

for EPOCH in range(epoch):

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_mask = train_input_masks[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        batch_segment = train_segment_ids[i: index]
        batch_segment = pad_sequences(batch_segment, padding='post')
        batch_y = pad_sequences(train_Y[i: index], padding='post')
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.input_masks: batch_mask,
                model.segment_ids: batch_segment
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = pad_sequences(test_Y[i: index], padding='post')
        batch_mask = test_input_masks[i: index]
        batch_mask = pad_sequences(batch_mask, padding='post')
        batch_segment = test_segment_ids[i: index]
        batch_segment = pad_sequences(batch_segment, padding='post')
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.input_masks: batch_mask,
                model.segment_ids: batch_segment
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )

train minibatch loop:   0%|          | 0/12500 [00:00<?, ?it/s]

epoch: 0, training loss: 3.531583, training acc: 0.397134, valid loss: 2.287189, valid acc: 0.551373



train minibatch loop:   0%|          | 0/12500 [00:00<?, ?it/s]

epoch: 1, training loss: 2.028329, training acc: 0.590376, valid loss: 1.858868, valid acc: 0.615519



train minibatch loop:   0%|          | 0/12500 [00:00<?, ?it/s]

...

epoch: 17, training loss: 1.011306, training acc: 0.754584, valid loss: 1.594291, valid acc: 0.661916



train minibatch loop:  71%|███████▏  | 8918/12500 [50:40<20:47,  2.87it/s, accuracy=0.752, cost=1.04]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train minibatch loop:   0%|          | 0/12500 [00:00<?, ?it/s]

epoch: 18, training loss: 1.011215, training acc: 0.754533, valid loss: 1.592084, valid acc: 0.662477



test minibatch loop: 100%|██████████| 313/313 [00:39<00:00,  8.02it/s, accuracy=0.735, cost=1.49] 

epoch: 19, training loss: 1.011770, training acc: 0.754386, valid loss: 1.596818, valid acc: 0.661871



In [18]:
from tensor2tensor.utils import bleu_hook

In [23]:
results = []
for i in tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = test_input_ids[i: index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_y = pad_sequences(test_Y[i: index], padding='post')
    batch_mask = test_input_masks[i: index]
    batch_mask = pad_sequences(batch_mask, padding='post')
    batch_segment = test_segment_ids[i: index]
    batch_segment = pad_sequences(batch_segment, padding='post')
    feed = {
        model.X: batch_x,
        model.input_masks: batch_mask,
        model.segment_ids: batch_segment
    }
    p = sess.run(model.fast_result,feed_dict = feed)[:,0,:]
    result = []
    for row in p:
        result.append([i for i in row if i > 3 and i not in [101, 102]])
    results.extend(result)

100%|██████████| 313/313 [15:48<00:00,  3.03s/it]


In [25]:
rights = []
for r in test_Y:
    rights.append([i for i in r if i > 3 and i not in [101, 102]])

In [26]:
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.37003958